### building chatbot with multiple tools 

## simple bs4 web scraping tool

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from pydantic import AnyUrl
from langchain_text_splitters import RecursiveCharacterTextSplitter


def scraper(url: AnyUrl) -> str:
    """
    Docstring for scraper
    
    :param url: This function is just made for making a basic web scraping tool
    :type url: AnyUrl
    :return: The whole content of the website
    :rtype: str
    """
    loading = WebBaseLoader(str(url))
    loader = loading.load()
    
    text_splitter = RecursiveCharacterTextSplitter()
    splitted_docs = text_splitter.split_documents(loader)

    doc_content = []

    for doc in splitted_docs:
        doc_content.append(doc.page_content)
    
    answer = ','.join([doc for doc in doc_content])
    answer_string = str(answer)
    return answer_string

url = "https://docs.tavily.com/documentation/api-reference/endpoint/search"
answer = scraper(url=url)
answer

'Tavily Search - Tavily DocsSkip to main contentTavily Docs home pageSearch...⌘KSupportGet an API keyGet an API keySearch...NavigationAPI ReferenceTavily SearchHomeDocumentationSDKsExamplesFAQChangelogAPI PlaygroundCommunityBlogOverviewAboutQuickstartCredits & PricingRate LimitsAPI ReferenceIntroductionPOSTTavily SearchPOSTTavily ExtractPOSTTavily CrawlPOSTTavily MapTavily Research (Beta)GETUsageBest PracticesBest Practices for SearchBest Practices for ExtractBest Practices for CrawlAPI Key ManagementTavily MCP ServerTavily MCP ServerStreamingStreamingPartnershipsIBMMarketplacesSnowflakeIntegrationsLangChainVercel AI SDKLlamaIndexOpenAIGoogle ADKAnthropicn8nMakeAgent BuilderLangflowZapierTinesDifyComposioAgnoPydantic AIFlowiseAICrewAIStackAILegalSecurity & CompliancePrivacy PolicyHelpHelp CenterTavily Search CrawlerTavily Search CrawlerPython SDKPythonCopyAsk AIfrom tavily import TavilyClient\n\ntavily_client = TavilyClient(api_key="tvly-YOUR_API_KEY")\nresponse = tavily_client.search(

## arxiv tool for research papers

In [ ]:
from langchain_community.retrievers import ArxivRetriever


def get_information_about_research_papers(query: str):
    retriever = ArxivRetriever(
        load_max_docs=1,
        get_full_documents=True
    )
    
    documents = retriever.invoke(query)
    paper_content = documents[0].page_content
    return paper_content



AutoDroid: LLM-powered Task Automation in
Android
Hao Wen1, Yuanchun Li1,2,†, Guohong Liu1, Shanhui Zhao1,∗, Tao Yu1,∗,
Toby Jia-Jun Li3, Shiqi Jiang4, Yunhao Liu5, Yaqin Zhang1, Yunxin Liu1,2
1 Institute for AI Industry Research (AIR), Tsinghua University
2 Shanghai Artificial Intelligence Laboratory
3 Department of Computer Science and Engineering, University of Notre Dame
4 Microsoft Research
5 Global Innovation Exchange & Department of Automation, Tsinghua University
ABSTRACT
Mobile task automation is an attractive technique that aims
to enable voice-based hands-free user interaction with smart-
phones. However, existing approaches suffer from poor scala-
bility due to the limited language understanding ability and
the non-trivial manual efforts required from developers or end-
users. The recent advance of large language models (LLMs)
in language understanding and reasoning inspires us to re-
think the problem from a model-centric perspective, where
task preparation, comprehension,

now we will build a simple graph structure that has the power to call tools (the arxiv agent and the scraper tool), and it will follow the REact agent architecture. let's see how to build it 

query --> llm --> tool --> output ---> llm --> tool --> output --> llm --> llm_reasoing --> output --> end 

In [36]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="llama-3.1-8b-instant", api_key = groq_api_key)

result = llm.invoke("what is the use of langchain?")
result.content

'LangChain is an open-source, Python-based framework for building and deploying large language models (LLMs) in real-world applications. It provides a set of tools and libraries to help developers create, train, and integrate LLMs into their projects.\n\nHere are some of the key use cases and features of LangChain:\n\n1. **LLM Integration**: LangChain allows developers to integrate LLMs into their applications, enabling features like text generation, sentiment analysis, language translation, and more.\n2. **Model Training**: LangChain provides tools for training and fine-tuning LLMs, including support for popular models like BERT, RoBERTa, and T5.\n3. **Model Serving**: LangChain enables developers to deploy and serve LLMs in production environments, making it easy to integrate LLMs into web applications, APIs, and other systems.\n4. **Conversational AI**: LangChain provides a set of tools for building conversational AI applications, including support for dialogue management, intent de

In [ ]:
from typing import TypedDict, Optional, Literal

class AgentState(TypedDict):
    query: str
    url: Optional[str]
    action: Optional[Literal["scrape", "arxiv", "none"]]
    scraped_content: Optional[str]
    paper_content: Optional[str]
    output: Optional[str]


In [47]:
from langchain.tools import tool

@tool
def scrape_website(url: str) -> str:
    """
    scrape textual content from a website
    """
    return scraper(url)

@tool
def search_research_papers(query: str) -> str:
    """Fetch relevant arXiv paper content"""
    return get_information_about_research_papers(query)


def llm_router(state: AgentState) -> str:
    prompt = f"""
    You are an AI router.

Decide the best action:
- Use `scrape` if the query involves a website or URL
- Use `arxiv` if the query is about research papers or academic content
- Use  `none` if none of the above

Query: {state['query']}
URL: {state.get('url')}

Respond ONLY with one word: scrape or arxiv 
"""
    decision = llm.invoke(prompt).content.strip().lower()
    return {
        "action" : decision
    }    

In [51]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode([scrape_website, search_research_papers])

def route_action(state: AgentState):
    if state["action"] == "scrape":
        return "scrape"
    
    if state["action"] == "arxiv":
        return "arxiv"
    
    if state["action"] == "none":
        return "none"

def scraper_adapter(state: AgentState) -> dict:
    return {
        "scraped_content": state["messages"][-1].content
    }


def arxiv_adapter(state: AgentState) -> dict:
    return {
        "paper_content": state["messages"][-1].content
    }

def synthesize_node(state: AgentState) -> dict:
    parts = []

    if state.get("scraped_content"):
        parts.append("WEB CONTENT:\n" + state["scraped_content"][:2000])

    if state.get("paper_content"):
        parts.append("RESEARCH PAPER:\n" + state["paper_content"][:2000])

    return {
        "output": "\n\n".join(parts)
    }


from langgraph.graph import StateGraph, START, END

graph = StateGraph(AgentState)

graph.add_node("router", llm_router)
graph.add_node("tool_executor", tool_node)
graph.add_node("scrape", scraper_adapter)
graph.add_node("arxiv", arxiv_adapter)
graph.add_node("synthesizer", synthesize_node)

graph.add_edge(START, "router")

graph.add_conditional_edges(
    "router",
    route_action,
    {
        "scrape": "tool_executor",
        "arxiv": "tool_executor"
    }
)

graph.add_edge("tool_executor", "scrape")
graph.add_edge("tool_executor", "arxiv")

graph.add_edge("scrape", "Synthesize")
graph.add_edge("arxiv", "Synthesize")

graph.add_edge("Synthesize", END)

app = graph.compile()



ValueError: Found edge starting at unknown node 'Synthesize'